In [1]:
import numpy 
import sys 
import nmslib 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [2]:
# Just a function to read sparse data
def read_data(filename, max_qty = None): 
    row = [] 
    col = [] 
    data = [] 
    read_qty = 0 
    with open(filename,'r') as f:  
        read_num_ft = 0
        for line in f: 
            x = line.replace(':', ' ').strip().split() 
            if (len(x) % 2 != 0):
                raise(Exception('Poorly formated line %d in file %s' % (read_qty + 1, filename)))
            if (len(x) == 0): continue
            for i in range(0, len(x), 2):     
                row.append(read_qty) 
                feat_id = int(x[i])
                read_num_ft = max(read_num_ft, feat_id + 1)
                col.append(feat_id) 
                data.append(float(x[i+1])) 

            read_qty = read_qty+1 
            if max_qty != None and read_qty >= max_qty: break
            
    print('Read %d rows, # of features %d' %  (read_qty, read_num_ft))
    ft_mat = csr_matrix((numpy.array(data), (numpy.array(row), numpy.array(col))), 
                         shape=(read_qty, read_num_ft)) 
    return (read_qty, ft_mat)

In [3]:
# Read data points
(all_qty, all_data_matrix) = read_data('../../sample_data/sparse_wiki_5K.txt') 

Read 5000 rows, # of features 100000


In [4]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(all_data_matrix, test_size = 0.1)

In [5]:
print("# of queries %d, # of data points %d"  % (query_matrix.shape[0], data_matrix.shape[0]) )

# of queries 500, # of data points 4500


In [6]:
# Set index parameters
# These are the most important onese
M = 30
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [7]:
# Number of neighbors 
K=100

In [8]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space='cosinesimil_sparse', data_type=nmslib.DataType.SPARSE_VECTOR) 
index.addDataPointBatch(data_matrix) 

4500

In [9]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'indexThreadQty': 4, 'post': 0, 'M': 30, 'efConstruction': 100}
Indexing time = 16.409487


In [10]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params) 

Setting query-time parameters {'efSearch': 100}


In [11]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=2.378762 (sec), per query=0.004758 (sec), per query adjusted for thread number=0.019030 (sec)


In [12]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(query_matrix)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.058578
brute-force kNN time total=2.372437 (sec), per query=0.004745 (sec)


In [13]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.970280


In [14]:
# Save a meta index
index.saveIndex('sparse_index.bin')

In [15]:
# Re-intitialize the library, specify the space, the type of the vector
newIndex = nmslib.init(method='hnsw', space='cosinesimil_sparse', data_type=nmslib.DataType.SPARSE_VECTOR) 

In [16]:
# For non-optimized indices we need to re-add data points
newIndex.addDataPointBatch(data_matrix) 

4500

In [17]:
# Re-load the index and re-run queries
newIndex.loadIndex('sparse_index.bin')

In [18]:
# Setting query-time parameters and querying
print('Setting query-time parameters', query_time_params)
newIndex.setQueryTimeParams(query_time_params)

query_qty = query_matrix.shape[0]
start = time.time() 
new_nbrs = newIndex.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

Setting query-time parameters {'efSearch': 100}
kNN time total=1.224607 (sec), per query=0.002449 (sec), per query adjusted for thread number=0.009797 (sec)


In [19]:
# Finally computing recall for the new result set
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(new_nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.970280
